# 데이터 살펴보기

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os

print(os.getcwd())
train = pd.read_csv("data/train_df_errno.csv")
test = pd.read_csv("data/test_df.csv")
sub = pd.read_csv("data/sample_submission.csv")
age = pd.read_csv("data/age_gender_info.csv")

train.shape, test.shape, sub.shape, age.shape

C:\Users\yanghj\Desktop\study\dacon_parking_demand_competition


((2896, 15), (1008, 14), (150, 2), (16, 23))

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2896 entries, 0 to 2895
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   단지코드        2896 non-null   object 
 1   총세대수        2896 non-null   int64  
 2   임대건물구분      2896 non-null   object 
 3   지역          2896 non-null   object 
 4   공급유형        2896 non-null   object 
 5   전용면적        2896 non-null   float64
 6   전용면적별세대수    2896 non-null   int64  
 7   공가수         2896 non-null   float64
 8   자격유형        2896 non-null   object 
 9   임대보증금       2327 non-null   object 
 10  임대료         2327 non-null   object 
 11  10분내지하철수    2685 non-null   float64
 12  10분내버스정류장수  2892 non-null   float64
 13  단지내주차면수     2896 non-null   float64
 14  등록차량수       2896 non-null   float64
dtypes: float64(6), int64(2), object(7)
memory usage: 339.5+ KB


In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   단지코드        1008 non-null   object 
 1   총세대수        1008 non-null   int64  
 2   임대건물구분      1008 non-null   object 
 3   지역          1008 non-null   object 
 4   공급유형        1008 non-null   object 
 5   전용면적        1008 non-null   float64
 6   전용면적별세대수    1008 non-null   int64  
 7   공가수         1008 non-null   float64
 8   자격유형        1006 non-null   object 
 9   임대보증금       828 non-null    object 
 10  임대료         828 non-null    object 
 11  10분내지하철수    970 non-null    float64
 12  10분내버스정류장수  1008 non-null   float64
 13  단지내주차면수     1008 non-null   float64
dtypes: float64(5), int64(2), object(7)
memory usage: 110.4+ KB


# 결측치가 얼마나 될까?

In [11]:
### 결측치를 확인
train.isna().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            0
임대보증금         569
임대료           569
10분내지하철수      211
10분내버스정류장수      4
단지내주차면수         0
등록차량수           0
dtype: int64

In [12]:
test.isna().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            2
임대보증금         180
임대료           180
10분내지하철수       38
10분내버스정류장수      0
단지내주차면수         0
dtype: int64

* 임대보증금, 임대료, 지하철역수, 버스정류장수(train only), 자격유형(test only)

In [ ]:
train.head()

In [ ]:
train.columns

In [ ]:
test.head()

In [13]:
train['자격유형'].value_counts()

A    1756
D     569
H     155
J     103
C      95
I      49
E      37
L      33
K      33
N      30
B      21
G       9
F       3
M       2
O       1
Name: 자격유형, dtype: int64

In [14]:
mapping = { 'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':8, 'I':9, 'J':10, 'K':11, 'L':12, 'M':13, 'N':14, 'O':15  }
train['자격유형']=train['자격유형'].map(mapping).astype(int)

## Lasso 모델

In [ ]:
#train dataset의 train과 test로 나누기
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge

sel = ['총세대수', '전용면적', '전용면적별세대수', '공가수',  '단지내주차면수', '자격유형']

X = train[sel]
y = train['등록차량수']


X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error

#평가해보기
'''
MAE(mean absolute error) : 각각의 값에 절대값을 취해서 이를 전부 더한 후, 갯수로 나누어주기
MSE(mean squared error) : (실제값 - 예측값 ) **2 전부 더해서 개수로나누어 준다
RMSE(root mean squared error) : MSE에 루트 씌워준다
'''
alpha = [0.0001,0.001,0.005,0.01,0.05,0.1,0.5,1]
for i in alpha:
    model = Lasso(alpha=i).fit(X_train, y_train)  #학습
    pred = model.predict(X_test) #예측
    pred

    MAE =  np.mean(abs(y_test - pred))
    MSE = mean_squared_error(y_test, pred)
    RMSE = mean_squared_error(y_test, pred) ** 0.5
    print("alpha:",i, "MAE:",MAE, "MSE:",MSE, "RMSE:",RMSE)



In [ ]:
#결정계수 구하기
print("학습(score):", model.score(X_train,y_train)) #결정계수
print("테스트(score):", model.score(X_test,y_test))

In [ ]:
#test에 예측한것을 붙여준다
test['등록차량수'] = pred